# Scheduler

There are many situations where you may want to control the number of calcjobs that are running at the same time. For example:
- On the HPC cluster, user may has a limit on the maximum number of submissions that can be running at the same time.
- On the local workstation, user may want to limit the number of calcjobs that are running at the same time to avoid overloading the system.

## Managing the Scheduler

Start a scheduler with name `test`:
```
workgraph scheduler start test
```

Stop the scheduler:
```
workgraph scheduler stop test
```

Show the status of the scheduler:
```
workgraph scheduler status test
```

Show details of the processes submitted to the scheduler:
```
workgraph scheduler show test
```

Set the maximum number of calcjobs that can be running at the same time:
```
workgraph scheduler set-max-jobs test 5
```

Let's start a scheduler 

In [1]:
!workgraph scheduler start test --max-calcjobs 2 --max-processes 10

Starting the scheduler ...


Check the status of the scheduler:

In [2]:
!workgraph scheduler status test

Name                status      pk   waiting  running   calcjob  max_calcjobs max_processes
test                Running  84410       0         0         0         2         10      



## Example Usage

Let's walk through an example where we creates four WorkGraphs with five calcjobs each.


In [3]:
from aiida_workgraph import WorkGraph
from aiida import load_profile, orm
from aiida.calculations.arithmetic.add import ArithmeticAddCalculation

load_profile()

# Use the calcjob: ArithmeticAddCalculation
code = orm.load_code("add@localhost")

for i in range(4):
    wg = WorkGraph("test_max_number_jobs")
    # Create N tasks
    for i in range(5):
        temp = wg.add_task(ArithmeticAddCalculation, name=f"add{i}", x=1, y=1, code=code)
        # Set a sleep option for each job (e.g., 10 seconds per job)
        temp.set({"metadata.options.sleep": 10})
    # submit the workgraph to a scheduler called "test-scheduler"
    wg.submit(scheduler="test")

WorkGraph process created, PK: 84411
WorkGraph process created, PK: 84412
WorkGraph process created, PK: 84413


04/14/2025 03:07:06 PM <1720302> aiida.engine.utils: [WARNING] Failed to write global variable `process|state_change|work` to `2025-04-14T15:07:06.472336+02:00` because the database was locked. If the storage plugin being used is `core.sqlite_dos` this is to be expected and can be safely ignored.


WorkGraph process created, PK: 84422


Note, all the WorkGraphs are submitted to a scheduler named `test`. Now, you can check the progress of the Scheduler using the following command:



In [4]:
! workgraph scheduler show test

Report: Scheduler: test
   PK  Created    Process label                    Process State      Priorities
-----  ---------  -------------------------------  ---------------  ------------
84411  7s ago     WorkGraph<test_max_number_jobs>  ⏵ Waiting
84412  6s ago     WorkGraph<test_max_number_jobs>  ⏵ Waiting
84413  5s ago     WorkGraph<test_max_number_jobs>  ⏹ Created                  -2
84416  5s ago     ArithmeticAddCalculation         ⏵ Waiting
84419  5s ago     ArithmeticAddCalculation         ⏵ Waiting
84423  5s ago     ArithmeticAddCalculation         ⏹ Created                  -3
84422  5s ago     WorkGraph<test_max_number_jobs>  ⏹ Created                   0
84426  4s ago     ArithmeticAddCalculation         ⏹ Created                   0
84429  4s ago     ArithmeticAddCalculation         ⏹ Created                   0
84432  3s ago     ArithmeticAddCalculation         ⏹ Created                  -1
84435  3s ago     ArithmeticAddCalculation         ⏹ Created                  -1
844


This command will display information about the currently running calcjobs, including the fact that there are a maximum of 2 calcjobs running simultaneously.

Additionally, you can monitor the progress visually by visiting `http://127.0.0.1:8000/scheduler/`, and go to the detail page of the scheduler by clicking on the scheduler name. This will show you the status of the scheduler.

![Scheduler](../_static/images/web-scheduler.png)



## WorkChain Support in the Scheduler

The scheduler in `aiida-workgraph` can also be used with WorkChains. However, this requires two additional steps compared to standard WorkChain usage:

1. Use the `submit_to_scheduler` function from `aiida_workgraph.utils.control`.
2. Override the `submit` method of the WorkChain you want to use.


### Example: Custom `MultiplyAddWorkChain`

Here we override the `submit` method of the `aiida.workflows.arithmetic.multiply_add.MultiplyAddWorkChain` to ensure it uses the scheduler for submitting processes.

```python
from aiida.orm import ProcessNode
from typing import Type, Any

class MultiplyAddWorkChain:
    """WorkChain to multiply two numbers and add a third, adapted for scheduling."""
    
    # other methods and attributes...

    def submit(
        self,
        process: Type["Process"],
        inputs: dict[str, Any] | None = None,
        **kwargs,
    ) -> ProcessNode:
        """Submit a process inside the workchain via the scheduler."""
        from aiida_workgraph.utils.control import submit_to_scheduler_inside_workchain
        return submit_to_scheduler_inside_workchain(self, process, inputs, **kwargs)
```


### Submit via Scheduler

You can now submit the patched WorkChain using:

```python
from aiida_workgraph.utils.control import submit_to_scheduler

x = 1
y = 2
z = 3
submit_to_scheduler(
    MultiplyAddWorkChain,
    inputs={"x": x, "y": y, "z": z, "code": code},
    scheduler="test"
)
```


### ⚠️ Warning: Nested WorkChains

**If the WorkChain calls other WorkChains internally (i.e. nested WorkChains), this approach will not work out of the box**. You must also:

- Patch all nested WorkChains in the same way (override their `submit` method).
- Restart the AiiDA daemon.

Failure to do so can result in processes that are not properly tracked or submitted via the scheduler.


## Persistent Scheduler

Last but not least, the scheduler is **persistent**. You can stop and restart it at any time using the same scheduler name, and all associated information will be preserved automatically.